# 2. Pulse Rate Estimation Algorithm

### Contents
- The Project description 
- The Code 


### Dataset
The [Troika](https://ieeexplore.ieee.org/document/6905737) dataset is used to build the algorithm. 

-----

## 2.1 Project description

**Code description**: 
This code is implemented to process files containing PPG and (3D) accelerometer signals, and use them to train a regression model for predicting the heart rate of a subject with an estimation error below 10 BPMs.
The code is divided in functions to facilitate its implementation and use. The code includes an accessing and loading stage of the signals, a preprocessing to extract relevant features and identify the targets. Then, all data is used to train a regression model, which is saved. To evaluate the algorithm, a cycle is put in place and each signal is used to estimate the error of the model.
To run the complete code it is necessary to call the Evaluate function with the sample frequency as an argument. As an output, the mean BPM error estimation is printed, and the regression model is available as 'regression_model.joblib'. The following sections describe in more depth the data used to train and test the model, each function included in the code to process the data, built the model and test its accuracy, and the method used to assess the performance of the model. 


**Data description**: 
From the original files, one PPG signal and three-axis acceleration signals were used. The signals were recorded simultaneously from subjects age 18 to 35. 12 files were included and the PPG signals were recorded from wrist by two pulse oximeters with green LEDs (wavelength: 515nm). The acceleration signal was also recorded from wrist by a three-axis accelerometer. Both the pulse oximeter and the accelerometer were embedded in a wristband. All signals were sampled at 125Hz and sent to a nearby computer via Bluetooth. 

Each dataset with the similar name 'DATA_01_TYPE01' contains a variable 'sig'. The first row is a simultaneous recording of a PPG signal, which is recorded from the wrist of each subject. The last three rows are simultaneous recordings of acceleration data (in x-, y-, and z-axis). During data recording, each subject ran on a treadmill with changing speeds. 

    For datasets with names containing 'TYPE01', the running speeds changed as follows:
        rest(30s) -> 8km/h(1min) -> 15km/h(1min) -> 8km/h(1min) -> 15km/h(1min) -> rest(30s)

    For datasets with names containing 'TYPE02', the running speeds changed as follows:
        rest(30s) -> 6km/h(1min) -> 12km/h(1min) -> 6km/h(1min) -> 12km/h(1min) -> rest(30s)

For each dataset, the calculated "ground-truth" heart rate is provided, stored in the datasets with the corresponding name, say 'REF_01_TYPE01'. In each of this kind of datasets, there is a variable 'BPM0', which gives the BPM value in every 8-second time window. Note that two successive time windows overlap by 6 seconds. Thus, the first value in 'BPM0' gives the calcualted heart rate "ground-truth" in the first 8 seconds, while the second value in 'BPM0' gives the calculated heart rate "ground-truth" from the 3rd second to the 10th second.

It is important to consider potential problems with the signal and the provided data, in general, such as the inclusion of incomplete information or incorrect information, for example due to biological and external noise sources, problems with the sensor and any device involved, etc. 


**Algorithhm description**: 
The algorithm is composed by several functions, organized in a sequence to load the original signals from stored files, processed these signals, trained a regression model and obtained predictive errors and confidence from test data. The functions building this algorithm are: 

* LoadTroikaDataset: Retrieve the .mat filenames for the troika dataset.
* LoadTroikaDataFile: Load and extract signals from a troika data file. 
* GenerateFeatures: Extract features from the dataset. Generate features by sliding a window across each dataset and computing the features on each window. 
* Featurize: Featurization of the accelerometer signal.
* LoadRegressor: Train and save a Decision Tree Regressor boost with an AdaBoost regressor, which is a meta-estimator that begins by fitting a regressor on the original dataset and then fits additional copies of the regressor on the same dataset but where the weights of instances are adjusted according to the error of the current prediction.
* RunPulseRateAlgorithm: Load the trained regression model and process the data to calculate the error and confidence for each feature extracted from test signals.
* AggregateErrorMetric: Computes an aggregate error metric based on confidence estimates.

To obtain the PPG signals, a photoplethysmogram optically measures blood flow at the wrist. The LEDs in a PPG sensor shine a typically green light into your skin and your red blood cells absorb that green light. The reflected light is then measured by the photodetector. When your heart beats and blood perfuses through the wrist, there are more red blood cells that absorb the green light and the photodetector sees a smaller signal. As the heart fills back up with blood and blood leaves your wrist, the more green light is reflected back and the photodetector reading goes up. Ans is this oscillating waveform that have been used to calculate the pulse rate.

The PPG signal is subject to different factors affecting the signal. Skin melanin content for example, impact the PPG signal since skin absorbs light from the LEDs as well and darker skin absorbs more light. This causes a DC shift and a reduction in SNR in the PPG signal for people with darker skin. The arm motion and position also affect the PPG signal due to the related movement of blood. Finger movement causes tendons and other structures to move, changing the sensor reading. Other potential sources of disturbance to the signal are malfunctioning of sensors, technical problems during the measurements, etc. Concretely, the project includes a pre-processing of the PPG signal, in which 3D acceleremeter data is used to partially "clean" the PPG data from the influence of movement and cadence of the arm during measurements. 

The main output of the algorithm (MAE) should be carefully assessed, taking into account potential problems (incorrect or incomplete information) or bias of the used data. Additionally, functions and parameters for each of them should be implemented correctly (format, data type, etc.) to avoid usage issues. 

**Algorithm performance**: 
The regresssion model was train/test using the LeaveOneGroupOut function, which provides train/test indices to split data according to a third-party provided group, in this case the original signal ID. This group information can be used to encode arbitrary domain specific stratifications of the samples as integers. The model is a decision tree regressor boost with an AdaBoost regressor, which is a meta-estimator that begins by fitting a regressor on the original dataset and then fits additional copies of the regressor on the same dataset but where the weights of instances are adjusted according to the error of the current prediction. As such, subsequent regressors focus more on difficult cases. Concretely, the model has 2000 estimators (the maximum number of estimators at which boosting is terminated. In case of perfect fit, the learning procedure is stopped early) and a maximum depth of the tree of 20. 
Then, the algorithm calculates the errors (a numpy array of errors between pulse rate estimates and corresponding reference heart rates) and the confidence (a numpy array of confidence estimates for each pulse rate error). These two variables are further used to computes the aggregate error metric based on confidence estimates. A higher confidence means a better estimate, and the best 90% of the estimates are above the 10th percentile confidence. This value, which is the mean absolute error (MAE) at 90% availability, is the ultimate output of the algorithm. 

## 2.2 Code

In [7]:
# Importing packages

import glob
import numpy as np
import scipy as sp
import pandas as pd
import scipy.io
import scipy.signal
from scipy.signal import savgol_filter
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import LeaveOneGroupOut
import joblib


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    
    return data_fls, ref_fls


def LoadTroikaDataFile(data_fl, ref_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    sig = data[2:]
    ref = scipy.io.loadmat(ref_fl)["BPM0"]
        
    return sig, ref


def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))


def Evaluate(fs):
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    fs = fs
    
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    
    sub_index, signals, target = [], [], []
 
    for i in range(len(data_fls)):
        signal, reference = LoadTroikaDataFile(data_fls[i], ref_fls[i])
        _id = data_fls[i].replace('./datasets/troika/training_data/', '') 
        sub_index.append(_id.replace('.mat', ''))
        signals.append(signal)
        target.append(reference)
    
    targets, sub_id, features, samples = GenerateFeatures(sub_index, signals, target, fs, 8, 2)  

    LoadRegressor(2000, 20, features, targets, sub_id)

    # Compute pulse rate estimates and estimation confidence
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        error, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(error)
        confs.append(confidence)
        print(data_fl)
        # Compute aggregate error metric
    
    errs = np.hstack(errs)
    confs = np.hstack(confs)

    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays
    return AggregateErrorMetric(errs, confs)


def RunPulseRateAlgorithm(data_fl, ref_fl):
    """
    Load the trained regression model and process the data to calculate the error and confidence for each feature extracted 
    from test signals

    Args:
        data_fl: a list of all filepaths to troika .mat files containing the signal data
        ref_fls: a list of all filepaths to .mat files containing reference data
        
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
        
    Returns:
        error: A numpy array containing the calculated error for each extracted feature
        confid: A numpy array containing the calculated confidence for each extracted feature
    """  
    fs = 125 # Sample frequency
    error, confid = [], []
    
    signal, reference = LoadTroikaDataFile(data_fl, ref_fl)
    _id = data_fl.replace('./datasets/troika/training_data/', '') 
    sub_index = _id.replace('.mat', '')
    
    targets, sub_id, features, samples = GenerateFeatures(sub_index, [signal], [reference], fs, 8, 2) 

    # Load the trained Random Forest Regressor model
    regression = joblib.load('regression_model.joblib')

    
    for i, features in enumerate(features):
        estimation = regression.predict(np.reshape(features, (1, -1)))
          
        ppg, accx, accy, accz = signal
        ppg = BandpassFilter(ppg, fs, 3, (1.67, 3.17))
        ppg = sp.signal.resample(ppg, 300*fs)
        ppg = ppg[samples[i]: samples[i] + 1000]
        
        freqs_ppg = np.fft.rfftfreq(len(ppg), 1/fs)
        fft_val = np.abs(np.fft.rfft(ppg))
        ppg_peak_freq = freqs_ppg[np.argmax(fft_val)]
        
        fft_val[freqs_ppg <= 40/60.0] = 0.0
        fft_val[freqs_ppg >= 240/60.0] = 0.0
    
        # Max magnitude frequency
        est_fs = estimation / 60.0
        fs_win = 30  / 60.0
        fs_win_e = (freqs_ppg >= est_fs - fs_win) & (freqs_ppg <= est_fs + fs_win)
        conf = np.sum(fft_val[fs_win_e])/np.sum(fft_val)
        
        error.append(float(np.abs((estimation - targets[i]))))
        confid.append(conf)
        
    return np.array(error), np.array(confid)



def BandpassFilter(signal, fs, order, passband):
    """
    Band pass filter

    Args:
        signal: Signal to be filtered
        fs: Sample frequency of the signal
        order: Order of the filter
        passband: A tuple containing the limits of the pass band as (min, max)
        
    Returns:
        sp.signal.filtfilt(b, a, signal): Filtered signal
    """
    b, a = sp.signal.butter(order, passband, btype='bandpass', fs=fs)
    
    return sp.signal.filtfilt(b, a, signal)


def LoadRegressor(n_estimators, max_depth, features, targets, sub_id):
    """
    Train and save a Random Forest Regressor

     Args:
        n_estimators: (number) The number of trees in the forest
        max_tree_depth: (number) The maximum depth of the tree
        features: (np.array) The feature matrix
        targets: (np.array) Class labels
        sub_id: (np.array) The subject id that the datapoint came from
        
    Returns:
        Saved regressor as 'regression_model.joblib'
    """
    
    regression = AdaBoostRegressor(DecisionTreeRegressor(max_depth=max_depth), learning_rate=0.0001, n_estimators=n_estimators, random_state=42)
    logo =  LeaveOneGroupOut() 
    
    scores = []

    for train_ind, test_ind in logo.split(features, targets.ravel(), sub_id):
        X_train, y_train = features[train_ind], targets[train_ind]
        X_test, y_test = features[test_ind], targets[test_ind]
        
        regression.fit(X_train, y_train.ravel())
        y_pred = regression.predict(X_test)
        score = mean_absolute_error(y_test.ravel(), y_pred)
        scores.append(score)
        
        # save
        joblib.dump(regression, 'regression_model.joblib')
        

def GenerateFeatures(subjects, signals, target, fs, window_length_s, window_shift_s):
    """Extract features from the dataset.
    
    Generate features by sliding a window across each dataset and computing
    the features on each window.
    
    Args:
    subjects: Features extracted from signals used for training the model
    signals: The sampling rate of the data
    target: Values extracted from reference data used as target value in supervised learning 
    fs: (number) Sample frequency
    window_length_s: (number) The length of the window in seconds
    window_shift_s: (number) The amount to shift the window by
    
    Returns:
    targets: (np.array) Class labels
    sub_id: (np.array) The subject id that the datapoint came from
    features: (np.array) The feature matrix
    samples: (np.array) Starting location of each window
    """
    window_length = window_length_s * fs
    window_shift = window_shift_s * fs
    feature_ppg_peak, feature_ppg_peak, features_l2, features_x, features_y, features_z, targets, sub_id, samples = [], [], [], [], [], [], [], [], []  
    mn_p, mn_l2, mn_x, mn_y, mn_z = [], [], [], [], []
    
    for index in range(len(signals)):

        ppg, accx, accy, accz = signals[index]   
        
        filt_ppg = BandpassFilter(ppg, fs, 3, (1.67, 3.17))
        filt_accx = BandpassFilter(accx, fs, 3, (1.67, 3.17))
        filt_accy = BandpassFilter(accy, fs, 3, (1.67, 3.17))
        filt_accz = BandpassFilter(accz, fs, 3, (1.67, 3.17))
    
        ppg_res = sp.signal.resample(filt_ppg, 300*fs)
        accx_res = sp.signal.resample(filt_accx, 300*fs)
        accy_res = sp.signal.resample(filt_accy, 300*fs)
        accz_res = sp.signal.resample(filt_accz, 300*fs)
    
        
        # Vector magnitude of the force on the accelerometer in 3D space
        acc_l2 = np.sqrt(accx_res**2 + accy_res**2 + accz_res**2)
        
        target_res = sp.signal.resample(target[index], 146)
        j = 0

        for i in range(0, len(ppg_res) - window_length, window_shift):
        
            # Selection of the time window
            ppg_ind = ppg_res[i: i + window_length]
            acc_l2_ind = acc_l2[i: i + window_length]
            accx_ind = accx_res[i: i + window_length]
            accy_ind = accy_res[i: i + window_length]
            accz_ind = accz_res[i: i + window_length]
        
            
            mn_p.append(np.mean(ppg_ind))
            mn_l2.append(np.mean(acc_l2_ind))
            mn_x.append(np.mean(accx_ind))
            mn_y.append(np.mean(accy_ind))
            mn_z.append(np.mean(accz_ind))
    
            ppg_peak, acc_l2_peak = Featurize(ppg_ind, acc_l2_ind, fs=fs)
            feature_ppg_peak.append(ppg_peak*60)
            features_l2.append(ppg_peak*60)
    
            ppg_peak, acc_x_peak = Featurize(ppg_ind, accx_ind, fs=fs)
            features_x.append(acc_x_peak*60)
            
            ppg_peak, acc_y_peak = Featurize(ppg_ind, accy_ind, fs=fs)
            features_y.append(acc_y_peak*60)
            
            ppg_peak, acc_z_peak = Featurize(ppg_ind, accz_ind, fs=fs)
            features_z.append(acc_z_peak*60)
            
            targets.append(target_res[j])
            sub_id.append(subjects[index])
            samples.append(i)
            j += 1
        
    # Smooth the signals
    features_ppg = savgol_filter(np.array(feature_ppg_peak), 51, 3) 
    features_l2 = savgol_filter(np.array(features_l2), 51, 3) 
    features_x = savgol_filter(np.array(features_x), 51, 3) 
    features_y = savgol_filter(np.array(features_y), 51, 3) 
    features_z = savgol_filter(np.array(features_z), 51, 3)
    
    features = np.array([features_ppg, features_l2, features_x, features_y, features_z, mn_p, mn_l2, mn_x, mn_y, mn_z]).T
    targets = np.array(targets)
    sub_id = np.array(sub_id)
    
    
    return targets, sub_id, features, samples


def Featurize(ppg, acc, fs):
    
    """Featurization of the signals
    
    Args:
    ppg: (np.array) ppg signal of the photoplethysmographic sensor
    acc_l2: (np.array) Vector magnitude of the force on the accelerometer in 3D space
    fs: (number) the sampling rate of the signals
    
    Returns:
        ppg_peak: The frequency peak of the ppg signal
        acc_peak: The frequency peak of the acc signal
    """
    # The mean of each channel
    freqs_ppg = np.fft.rfftfreq(len(ppg), 1/fs)
    fft_val = np.abs(np.fft.rfft(ppg))
    ppg_peak_freq = freqs_ppg[np.argmax(fft_val)]
    
    freqs_acc = np.fft.rfftfreq(len(acc), 1/fs)
    fft_acc = np.abs(np.fft.rfft(acc))
    acc_peak_freq = freqs_acc[np.argmax(fft_acc[10:])] # Ignore DC component

    if np.abs(ppg_peak_freq - acc_peak_freq) <= 10:
        ppg_peak_freq = freqs_ppg[np.argsort(fft_val, axis=0)][-2]
    
    ppg_peak = ppg_peak_freq
    acc_peak = acc_peak_freq

    return ppg_peak, acc_peak

In [8]:
Evaluate(125)

./datasets/troika/training_data/DATA_01_TYPE01.mat
./datasets/troika/training_data/DATA_02_TYPE02.mat
./datasets/troika/training_data/DATA_03_TYPE02.mat
./datasets/troika/training_data/DATA_04_TYPE01.mat
./datasets/troika/training_data/DATA_04_TYPE02.mat
./datasets/troika/training_data/DATA_05_TYPE02.mat
./datasets/troika/training_data/DATA_06_TYPE02.mat
./datasets/troika/training_data/DATA_07_TYPE02.mat
./datasets/troika/training_data/DATA_08_TYPE02.mat
./datasets/troika/training_data/DATA_10_TYPE02.mat
./datasets/troika/training_data/DATA_11_TYPE02.mat
./datasets/troika/training_data/DATA_12_TYPE02.mat


2.6589510919563688